In [3]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [36]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection to match dimensions
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # Add the shortcut connection
        out = F.relu(out)
        return out

In [ ]:
"""class ResNet(nn.Module):
    def __init__(self,block, num_blocks, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_Channels = 64
        self.conv1 = nn.Conv2d(1,64,kernel_size = 3,stride = 1,padding = 1,bias = False) 
        self.bn1 = nn.BatchNorm2d(64) 
        self.layer1 = self._make_layer(block,64,num_blocks[0],stride = 1)
        self.layer2 = self._make_layer(block,128,num_blocks[0],stride = 2)
        self.layer3 = self._make_layer(block,256,num_blocks[0],stride = 2)
        self.layer4 = self._make_layer(block,512,num_blocks[0],stride = 2)

    def _make_layer(self, block, out_channels , nums_blocks , stride)    :
        strides = [stride] + [1]*(num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)    

    def forward(self, x)    :
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out) 
        return out"""

In [37]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels  # Update in_channels for the next block
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)  # Global average pooling
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [38]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [32]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root ='./data',train = True,download = True, transform = transform)
test_dataset = datasets.MNIST(root ='./data',train = False,download = True, transform = transform)

train_loader = DataLoader(dataset = train_dataset, batch_size = 128, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 128, shuffle = False)

device = torch.device("cude" if torch.cuda.is_available() else "cpu")


In [39]:
model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')            

In [40]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [ ]:
for epoch in range(1, 11):
    train(epoch)
    test()

Train Epoch: 1 [0/60000] Loss: 2.342106
Train Epoch: 1 [12800/60000] Loss: 0.063598
Train Epoch: 1 [25600/60000] Loss: 0.044447
Train Epoch: 1 [38400/60000] Loss: 0.030978
Train Epoch: 1 [51200/60000] Loss: 0.024900

Test set: Average loss: 0.0004, Accuracy: 9833/10000 (98%)

Train Epoch: 2 [0/60000] Loss: 0.093195
Train Epoch: 2 [12800/60000] Loss: 0.022739
Train Epoch: 2 [25600/60000] Loss: 0.016875
Train Epoch: 2 [38400/60000] Loss: 0.021701
Train Epoch: 2 [51200/60000] Loss: 0.053440

Test set: Average loss: 0.0003, Accuracy: 9893/10000 (99%)

Train Epoch: 3 [0/60000] Loss: 0.041875
Train Epoch: 3 [12800/60000] Loss: 0.006729
Train Epoch: 3 [25600/60000] Loss: 0.033374
Train Epoch: 3 [38400/60000] Loss: 0.007544
Train Epoch: 3 [51200/60000] Loss: 0.002264

Test set: Average loss: 0.0003, Accuracy: 9893/10000 (99%)

Train Epoch: 4 [0/60000] Loss: 0.025178
Train Epoch: 4 [12800/60000] Loss: 0.005857
Train Epoch: 4 [25600/60000] Loss: 0.012894
Train Epoch: 4 [38400/60000] Loss: 0.0647